# Build and deploy a model with custom Docker Images

In this example, we will deploy inference servers on customized Docker images using Azure Secure Container Registry. We will extend a pre-built image from Azure's curated image library and build an image from base Ubuntu 18.04. 

## Prerequisites

* To use Azure Machine Learning, you must have an Azure subscription. If you don't have an Azure subscription, create a free account before you begin. Try the [free or paid version of Azure Machine Learning](https://azure.microsoft.com/free/).

* Install and configure the [Python SDK v2](sdk/setup.sh).

* You must have an Azure resource group, and you (or the service principal you use) must have Contributor access to it.

* You must have an Azure Machine Learning workspace. 

* You must have an Azure Secure Container registry. One is created automatically created for a workspace without one upon first usage, however in this example we explicitly reference the container registry by name, so you need it beforehand. You can create one through the Azure Portal. 

## Initial set up

We will first get a handle to the workspace, which will be reused later as we deploy images. You must already have an existing Azure Secure Container Registry associated with the workspace.

In [1]:
subscription_id = '6fe1c377-b645-4e8e-b588-52e57cc856b2'
resource_group = 'v-alwallace-test'
workspace = 'valwallace'
container_registry_name = 'valwallaceskr'

In [2]:
from azure.ml import MLClient
from azure.ml.entities import ManagedOnlineDeployment, ManagedOnlineEndpoint
from azure.identity import DefaultAzureCredential
from random import randint

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

## Basic Docker image deployment

### Define deployment and container registry details

The name of the deployment, container registry, and container name are all required. We will create a new container using the name here, however, The endpoint name is optional, the code below will generate a random name likely to be unique within the region.

In [3]:
# Required
deployment_name = 'docker-basic'
container_name = 'docker-basic'
# Optional
endpoint_name = f'docker-basic-{randint(1e3,1e7)}'

The first image we will build is the OpenMPI3.1.2 Ubuntu 18.04 image from Azure. This image contains all of the dependencies required to score the model, as well as the AML Inference Server, so our Dockerfile is trivial: 

```Dockerfile 
FROM mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest
```

To begin, we will build the image locally and test a local deployment. If you're rebuilding, pass the `--no-cache` flag. 

In [4]:
!docker build -t {container_name} docker_basic/. 

[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (2/2)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
[+] Building 0.2s (5/5) FINISHED                                                
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for mcr.microsoft.com/azureml/minimal-ubuntu  0.0s
 => CACHED [1/1] FROM mcr.microsoft.c

The image is now among your local images, which you can see by running the command  `docker image list` or `docker image ls`. The image is now ready to be included in a deployment, however, let's run the image now and see the AML Inference Server load. It comes preloaded in most of the Azure-curated images.  Since there are are no models and no scoring script provided to it yet, it will exit quickly. 

In [5]:
!docker run -t {container_name}

2022-04-20T17:39:38,346349041+00:00 - rsyslog/run 
2022-04-20T17:39:38,346439343+00:00 - gunicorn/run 
2022-04-20T17:39:38,347756537+00:00 | gunicorn/run | 
2022-04-20T17:39:38,349158728+00:00 | gunicorn/run | ###############################################
2022-04-20T17:39:38,350396320+00:00 | gunicorn/run | AzureML Container Runtime Information
2022-04-20T17:39:38,351563140+00:00 | gunicorn/run | ###############################################
2022-04-20T17:39:38,353014974+00:00 | gunicorn/run | 
2022-04-20T17:39:38,353315738+00:00 - nginx/run 
2022-04-20T17:39:38,357228326+00:00 | gunicorn/run | 
2022-04-20T17:39:38,360816232+00:00 | gunicorn/run | AzureML image information: minimal-ubuntu18.04-py37-cpu-inference:20220419.v1
2022-04-20T17:39:38,362060015+00:00 | gunicorn/run | 
2022-04-20T17:39:38,363385238+00:00 | gunicorn/run | 
2022-04-20T17:39:38,364850028+00:00 | gunicorn/run | PATH environment variable: /opt/miniconda/envs/amlenv/bin:/opt/miniconda/bin:/usr/local/sbin:/usr/loc

In [6]:
!acr build image

/bin/bash: acr: command not found


Let's fix that by including the image in an environment through an YAML file, along with a trained model and `score.py` script for the Inference Server to call. This `deployment.yml` file specifies the trained model file under `model` as well as the scoring script under `code_configuration`.

```yaml 
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: deployment_name
endpoint_name: endpoint_name
model:
  path: './docker_basic/sklearn_regression_model.pkl'
code_configuration: 
  code: './docker_basic'
  scoring_script: 'score.py'
environment:
  image: container_name:latest
```

We will import the YAML file and update variables, however, the in your workloads the file can be directly loaded by passing the file path to the `.load` method of a `ManagedOnlineDeployment` object.

In [7]:
import yaml
with open('./docker_basic/deployment_local.yml','r') as f:
    deployment_yaml = yaml.safe_load(f)

In [21]:
deployment_yaml['name'] = deployment_name
deployment_yaml['endpoint_name'] = endpoint_name
deployment_yaml['environment']['image'] = f'{container_name}:latest'

In [9]:
deployment_yaml

OrderedDict([('$schema',
              'https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json'),
             ('name', 'docker-basic'),
             ('endpoint_name', 'docker-basic-9455673'),
             ('model',
              OrderedDict([('path',
                            './docker_basic/sklearn_regression_model.pkl')])),
             ('code_configuration',
              OrderedDict([('code', './docker_basic'),
                           ('scoring_script', 'score.py')])),
             ('environment', OrderedDict([('image', 'docker-basic:latest')])),
             ('instance_type', 'Standard_F2s_v2'),
             ('instance_count', 1)])

Now we can deploy. First we create an endpoint and then a deployment.

In [19]:
#!docker run -d -p 5000:5000 --restart=always --name registry registry:latest
#!docker tag localdomain:5000/docker-basic  docker-basic
# az acr login
!az acr build --image {container_name} --registry {container_registry_name} --file Dockerfile docker_basic/.

Packing source code into tar to upload...
Uploading archived source code from '/tmp/build_archive_7f023d29e8b74ab78759fd7d4f0730a4.tar.gz'...
Sending context (2.171 KiB) to registry: valwallaceskr...
Queued a build with ID: cht
Waiting for an agent...
2022/04/20 17:49:57 Downloading source code...
2022/04/20 17:49:59 Finished downloading source code
2022/04/20 17:49:59 Using acb_vol_4394860d-6489-4e3a-a60a-155354bb4c1d as the home volume
2022/04/20 17:49:59 Setting up Docker configuration...
2022/04/20 17:50:00 Successfully set up Docker configuration
2022/04/20 17:50:00 Logging in to registry: valwallaceskr.azurecr.io
2022/04/20 17:50:01 Successfully logged into valwallaceskr.azurecr.io
2022/04/20 17:50:01 Executing step ID: build. Timeout(sec): 28800, Working directory: '', Network: ''
2022/04/20 17:50:01 Scanning for dependencies...
2022/04/20 17:50:01 Successfully scanned dependencies
2022/04/20 17:50:01 Launching container with name: build
Sending build context to Docker daemon  8

In [14]:
deployment_yaml['environment']['image'] = f'localhost:5000/{container_name}:latest'

In [13]:
deployment_yaml

OrderedDict([('name', 'docker-basic'),
             ('endpoint_name', 'docker-basic-9455673'),
             ('model',
              OrderedDict([('path',
                            './docker_basic/sklearn_regression_model.pkl')])),
             ('code_configuration',
              OrderedDict([('code', './docker_basic'),
                           ('scoring_script', 'score.py')])),
             ('environment',
              OrderedDict([('image',
                            'localhost:5000/myadmin/docker-basic:latest')])),
             ('instance_type', 'Standard_F2s_v2'),
             ('instance_count', 1)])

In [22]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name)
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)
deployment = ManagedOnlineDeployment.load_from_dict(deployment_yaml)
deployment = ml_client.online_deployments.begin_create_or_update(deployment, local=True)

Updating local endpoint (docker-basic-9455673) .Done (0m 5s)
Creating local deployment (docker-basic-9455673 / docker-basic) .
Building Docker image from Dockerfile.......................
Step 1/5 : FROM docker-basic:latest
pull access denied for docker-basic, repository does not exist or may require 'docker login': denied: requested access to the resource is deniedDone (2m 0s)


LocalEndpointImageBuildError: Building the local endpoint image failed with error: pull access denied for docker-basic, repository does not exist or may require 'docker login': denied: requested access to the resource is denied

## Extend a curated Docker image

First, we will extend the  no-framework inference Docker image from [Azure's curated image library](/azure/machine-learning/concept-prebuilt-docker-images-inference). This image is built from a minimal Ubuntu 18.04 base image and does not include any frameworks such as Tensorflow or Torch, however, it does include the Azure Machine Learning Inference Server, which enables the rapid deployment of inference servers through a single `score.py` file that calls the scored model. Our working directory looks like this: 

```bash
model/
    sklearn_regression_model.pkl
environment/
    requirements.txt
code/
    score.py
```

Each of these directories will be copied into the image in the Dockerfile. The model directory contains the trained model object we will call to score each request. This path will be passed to the Inferencing Server, and may contain nested subdirectory trees corresponding to different models and verisons. The `score.py` file is located in the code directory. The inferencing server will call the score.py file from the relevant subdirectory depending on the model version, so there is no need for the score.py file to keep track of this tree. The requirements.txt file contains the additional Python packages we will install in the image. It looks like this: 

```
numpy==1.21.2
pip==21.2.4
scikit-learn==0.24.2
scipy==1.7.1
azureml-defaults==1.38.0
inference-schema[numpy-support]==1.3.0
joblib==1.0.1
```

For this basic deployment, we will be installing these packages in the default Python path, which is configured ahead of time to support the inferencing server. More robust configurations using virtualenvs or `conda` environments are possible. After image creation, requirements files can be dynamically loaded by the inferencing server or additional dependencies can be specified through an `Environment`. See the Environment and ManagedOnlineDeployment schemas for more details.

```dockerfile
FROM mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest
USER root:root
COPY environment /var/environment
RUN pip install -r /var/environment/requirements.txt
```

Next, we copy the model and code directories into the image, and make `ENV` variables to specify the scoring script and model directory. The Inferencing Server determines the correct entrypoints using these variables. 

USER dockeruser

COPY code /var/azureml-app
ENV AZUREML_ENTRY_SCRIPT=score.py

COPY model /var/azureml-app/azureml-models
ENV AZUREML_MODEL_DIR=/var/azureml-app/azureml-models

After the base image is referenced, we configure the additional packages and libraries required to score the model. For this base

# Extend an Azure curated Docker image

We begin by extending the frameworkless Azure image built on minimal Ubuntu 18.04. While it is a minimal build, it does come with the Azure Inferencing Server, so scored models can be easily integrated into the image by providing the usual `score.py` entrypoint. With a pre

In [ ]:
!az acr login --name {container_registry_name}

In [ ]:
!az acr build --image custom_container --registry {container_registry_name} --file Dockerfile .

## Create a managed online deployment

First, we deploy an online endpoint.

In [ ]:
ml_client.online_endpoints.begin_create_or_update(ManagedOnlineEndpoint(name='custom-container-9230'))

In [ ]:
deployment = ManagedOnlineDeployment.load('deployment.yml')

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment)

In [ ]:
auth_token = ml_client.online_endpoints.list_keys(endpoint_name).primary_key

In [ ]:
endpoint = ml_client.online_endpoints.get(endpoint_name)
scoring_uri = endpoint.scoring_uri

In [ ]:

response=None
import requests
import json 
with open(os.path.join('.','sample-request.json')) as f:
    data = json.loads(f.read())
headers = {}
headers = {'Authorization' : f'Bearer {auth_token}', 'Content-Type':'application/json'} 
#scoring_uri = "https://custom-container-9230.eastus2.inference.ml.azure.com/score"
response = requests.post(url=scoring_uri,
                        headers=headers,
                        data=json.dumps(data))

## Extend a curated Docker image

First, we will extend the  no-framework inference Docker image from [Azure's curated image library](/azure/machine-learning/concept-prebuilt-docker-images-inference). This image is built from a minimal Ubuntu 18.04 base image and does not include any frameworks such as Tensorflow or Torch, however, it does include the Azure Machine Learning Inference Server, which enables the rapid deployment of inference servers through a single `score.py` file that calls the scored model. Our working directory looks like this: 

```bash
model/
    sklearn_regression_model.pkl
environment/
    requirements.txt
code/
    score.py
```

Each of these directories will be copied into the image in the Dockerfile. The model directory contains the trained model object we will call to score each request. This path will be passed to the Inferencing Server, and may contain nested subdirectory trees corresponding to different models and verisons. The `score.py` file is located in the code directory. The inferencing server will call the score.py file from the relevant subdirectory depending on the model version, so there is no need for the score.py file to keep track of this tree. The requirements.txt file contains the additional Python packages we will install in the image. It looks like this: 

In [ ]:
```
numpy==1.21.2
pip==21.2.4
scikit-learn==0.24.2
scipy==1.7.1
azureml-defaults==1.38.0
inference-schema[numpy-support]==1.3.0
joblib==1.0.1
```

For this basic deployment, we will be installing these packages in the default Python path, which is configured ahead of time to support the inferencing server. More robust configurations using virtualenvs or `conda` environments are possible. After image creation, requirements files can be dynamically loaded by the inferencing server or additional dependencies can be specified through an `Environment`. See the Environment and ManagedOnlineDeployment schemas for more details.

```dockerfile
FROM mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest
USER root:root
COPY environment /var/environment
RUN pip install -r /var/environment/requirements.txt
```

Next, we copy the model and code directories into the image, and make `ENV` variables to specify the scoring script and model directory. The Inferencing Server determines the correct entrypoints using these variables. 

USER dockeruser

COPY code /var/azureml-app
ENV AZUREML_ENTRY_SCRIPT=score.py

COPY model /var/azureml-app/azureml-models
ENV AZUREML_MODEL_DIR=/var/azureml-app/azureml-models